In [3]:
# Fine-tune GPT-3.5 Turbo on posts from the subreddit r/AskReddit that are stored in the askreddit.csv file.
# The model is fine-tuned using the Azure OpenAI API.
# The fine-tuned model should be able to genearate reddit posts.

from openai import AzureOpenAI
import os
from dotenv import load_dotenv

# Get the credentials from the environment
load_dotenv()

# Initialize the Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-05-01-preview"  # This API version or later is required to access seed/events/checkpoint capabilities
)

# Upload the training dataset file to Azure OpenAI with the SDK
training_file_name = "askreddit.jsonl"
validation_file_name = "askreddit_post_validation.jsonl"

training_response = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response.id

validation_response = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)

Training file ID: file-bc908f3023ff422791bb1d5e28af06bd


In [7]:
# Start fine-tuning the model
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-35-turbo-0125"
)

# Motitor the fine-tuning job with job ID
job_id = response.id

print("Job ID:", job_id)
print("Job status:", response.status)
print(response.model_dump_json(indent=2))

Job ID: ftjob-ba4c1bd78a6d4f39a9a0e6eac074982b
Job status: pending
{
  "id": "ftjob-ba4c1bd78a6d4f39a9a0e6eac074982b",
  "created_at": 1724697711,
  "error": null,
  "fine_tuned_model": null,
  "finished_at": null,
  "hyperparameters": {
    "n_epochs": -1,
    "batch_size": -1,
    "learning_rate_multiplier": 1
  },
  "model": "gpt-35-turbo-0125",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": null,
  "seed": 840034487,
  "status": "pending",
  "trained_tokens": null,
  "training_file": "file-bc908f3023ff422791bb1d5e28af06bd",
  "validation_file": "file-a12e4990d66f445ba7b56906ff9719d5",
  "estimated_finish": null,
  "integrations": null
}


In [16]:
# Track training status

from IPython.display import clear_output
import time

start_time = time.time()

# Get the status of our fine-tuning job.
response = client.fine_tuning.jobs.retrieve(job_id)

status = response.status

# If the job isn't done yet, poll it every 10 seconds.
while status not in ["succeeded", "failed"]:
    time.sleep(10)

    response = client.fine_tuning.jobs.retrieve(job_id)
    print(response.model_dump_json(indent=2))
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = response.status
    print(f'Status: {status}')
    clear_output(wait=True)

print(f'Fine-tuning job {job_id} finished with status: {status}')

# List all fine-tuning jobs for this resource.
print('Checking other fine-tune jobs for this resource.')
response = client.fine_tuning.jobs.list()
print(f'Found {len(response.data)} fine-tune jobs.')

Fine-tuning job ftjob-ba4c1bd78a6d4f39a9a0e6eac074982b finished with status: succeeded
Checking other fine-tune jobs for this resource.
Found 1 fine-tune jobs.


In [18]:
# Retrieve fine_tuned_model name

response = client.fine_tuning.jobs.retrieve(job_id)

print(response.model_dump_json(indent=2))
fine_tuned_model = response.fine_tuned_model

{
  "id": "ftjob-ba4c1bd78a6d4f39a9a0e6eac074982b",
  "created_at": 1724697711,
  "error": null,
  "fine_tuned_model": "gpt-35-turbo-0125.ft-ba4c1bd78a6d4f39a9a0e6eac074982b",
  "finished_at": 1724704647,
  "hyperparameters": {
    "n_epochs": 3,
    "batch_size": 1,
    "learning_rate_multiplier": 1
  },
  "model": "gpt-35-turbo-0125",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": [
    "file-c572bf42fbb04804acc4cd950c8fc599"
  ],
  "seed": 840034487,
  "status": "succeeded",
  "trained_tokens": 88887,
  "training_file": "file-bc908f3023ff422791bb1d5e28af06bd",
  "validation_file": "file-a12e4990d66f445ba7b56906ff9719d5",
  "estimated_finish": null,
  "integrations": null
}


In [62]:
# Deploy the fine-tuned model

import json
import requests
import os

load_dotenv()

token = os.getenv("TEMP_AUTH_TOKEN")
subscription = os.getenv("AZURE_SUBSCRIPTION_ID")
resource_group = os.getenv("AZURE_RESOURCE_GROUP")
resource_name  = os.getenv("AZURE_OPENAI_RESOURCE_NAME")
model_deployment_name = "gpt-35-turbo-0125-askreddit-posts"
fine_tuned_model = os.getenv("FINE_TUNED_MODEL_POST")

deploy_params = {'api-version': "2023-05-01"}
deploy_headers = {'Authorization': 'Bearer {}'.format(token), 'Content-Type': 'application/json'}

deploy_data = {
    "sku": {"name": "standard", "capacity": 1},
    "properties": {
        "model": {
            "format": "OpenAI",
            "name": fine_tuned_model,
            "version": "1"
        }
    }
}

deploy_data = json.dumps(deploy_data)   # Convert the data to a JSON string

request_url = f'https://management.azure.com/subscriptions/{subscription}/resourceGroups/{resource_group} \
    /providers/Microsoft.CognitiveServices/accounts/{resource_name}/deployments/{model_deployment_name}'

print('Creating a new deployment...')

r = requests.put(request_url, params=deploy_params, headers=deploy_headers, data=deploy_data)

print(r)
print(r.reason)
print(r.json())

Creating a new deployment...
<Response [201]>
Created
{'id': '/subscriptions/868761f1-f48e-4aef-a9af-d397d405ff2d/resourceGroups/Diplomski_Rad/providers/Microsoft.CognitiveServices/accounts/DiplomskiRadJFP/deployments/gpt-35-turbo-0125-askreddit-posts', 'type': 'Microsoft.CognitiveServices/accounts/deployments', 'name': 'gpt-35-turbo-0125-askreddit-posts', 'sku': {'name': 'standard', 'capacity': 1}, 'properties': {'model': {'format': 'OpenAI', 'name': 'gpt-35-turbo-0125.ft-ba4c1bd78a6d4f39a9a0e6eac074982b', 'version': '1'}, 'versionUpgradeOption': 'NoAutoUpgrade', 'capabilities': {'maxContextToken': '16385', 'maxOutputToken': '4096', 'chatCompletion': 'true'}, 'provisioningState': 'Creating', 'rateLimits': [{'key': 'request', 'renewalPeriod': 10, 'count': 1}, {'key': 'token', 'renewalPeriod': 60, 'count': 1000}]}, 'systemData': {'createdBy': 'jprgin00@fesb.hr', 'createdByType': 'User', 'createdAt': '2024-08-26T23:10:39.3307203Z', 'lastModifiedBy': 'jprgin00@fesb.hr', 'lastModifiedByTyp